In [96]:
import os
os.chdir("F:/Airbnb") 

In [128]:
import pandas as pd
dc_listing = pd.read_csv("cleaned.csv")
print(dc_listing.shape)
dc_listing.head()

(3723, 8)


,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,160,1,1125,0
1,6,3.0,3.0,3.0,350,2,30,65
2,1,2.0,1.0,1.0,50,2,1125,1
3,2,1.0,1.0,1.0,95,1,1125,0
4,4,1.0,1.0,1.0,50,7,1125,0


In [129]:
import numpy as np
our_acc_value = 3
first_living_space_value = dc_listing.loc[0,'accommodates']
first_distance = np.abs(first_living_space_value - our_acc_value)
print(first_distance) 

1


In [131]:
dc_listing['distance']=np.abs(dc_listing.accommodates-our_acc_value)
dc_listing.distance.value_counts().sort_index() 

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [132]:
dc_listing=dc_listing.sample(frac=1,random_state=0)
dc_listing = dc_listing.sort_values('distance')
dc_listing.price.head()  

2645     75
2825    120
2145     90
2541     50
3349    105
Name: price, dtype: int64

In [133]:
mean_price = dc_listing.price.iloc[:5].mean()
mean_price  

88.0

In [134]:
dc_listing.drop('distance',axis=1)
train_df=dc_listing.copy().iloc[:2793]
test_df=dc_listing.copy().iloc[2793:] 

In [135]:
def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listing[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price) 

In [136]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')

In [137]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse 

212.7327304945243

In [138]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)

    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 212.7327304945243
RMSE for the bedrooms column: 210.2660045945898
RMSE for the bathrooms column: 217.00161041967218
RMSE for the number_of_reviews column: 244.69636668934777


In [139]:
normalized_listings=dc_listing
normalized_listings = normalized_listings.sample(frac=1,random_state=0)
norm_train_df = normalized_listings.copy().iloc[0:2793]
norm_test_df = normalized_listings.copy().iloc[2793:] 

In [140]:
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = normalized_listings.iloc[20][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
first_fifth_distance  

0.5

In [141]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price) 

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
rmse = norm_test_df['squared_error'].mean()
rmse = rmse ** (1/2)
print(rmse) 

110.723744382


In [142]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(algorithm="brute") 

In [143]:
norm_train_df = norm_train_df.fillna(norm_train_df.mean(),inplace=True) 

In [144]:
knn.fit(norm_train_df[cols], norm_train_df['price'])

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [145]:
norm_test_df = norm_train_df.fillna(norm_test_df.mean(),inplace=True)

In [146]:
two_features_predictions = knn.predict(norm_test_df[cols])

In [147]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(norm_test_df['price'], two_features_predictions)
two_features_rmse = two_features_mse ** (1/2)
print(two_features_rmse)

117.159666744


In [148]:
knn = KNeighborsRegressor(algorithm='brute')

cols = ['accommodates','bedrooms','bathrooms','beds']

knn.fit(norm_train_df[cols], norm_train_df['price'])
four_features_predictions = knn.predict(norm_test_df[cols])
four_features_mse = mean_squared_error(norm_test_df['price'], four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)
four_features_rmse

113.15614697523691